In [511]:
import pandas as pd
import pymatreader as pmr
import numpy as np

In [512]:
file_name='S01' #Указать номер человека#
data=pmr.read_mat(f'исходное/{file_name}.mat') #Указать путь к файлу#
df_not_clean=pd.DataFrame(data[file_name])

In [513]:
useless_demographic=['Age', 'Gender', 'DominantHand', 'Education', 'MaritalStatus', 'Children']
useless_profile=['WeeklySupermarketVisits', 'SupermarketVisitDuration', 'PriceImpact', 'BrandImpact', 'DiscountImpact', 'AdvertisementImpact', 'SuggestionImpact', 'ShoppingList', 'VerbalVisual', 'Spontaneous', 'VarietySeeker', 'UtilitarianMotivation', 'HedonicMotivation', 'Extraversion', 'Neuroticism', 'Agreeableness', 'Openness', 'Conscientiousness', 'BargainHunter']

In [514]:
df_not_clean.rename(index={
    'education': 'Education',
    'marital_status': 'MaritalStatus'
}, inplace=True)

In [515]:
useless_products=np.arange(1, 25, 1)
s=[]
for i in useless_products:
    s.append('Product'+str(i))
useless_products=s

In [516]:
et_useful_data=df_not_clean['ET_clean']
et_useful_data=et_useful_data.drop(useless_demographic+useless_profile+useless_products)
et_useful_data=pd.DataFrame(et_useful_data).transpose()

In [517]:
et_data=et_useful_data.Data['ET_clean']
et_chans=et_useful_data.chans['ET_clean']

In [518]:
df_products=df_not_clean.drop(columns=['Profile', 'Demographics', 'EEG_clean', 'ET_clean'])
df_products.drop(useless_demographic+useless_profile+['Data', 'Fs', 'chans'], inplace=True)
useless_pages=['Page1', 'Page2', 'Page3', 'Page4', 'Page5', 'Page6']
num_of_products=24

In [519]:
df_et=pd.DataFrame()

In [ ]:
#Как-будто не работает#
for t in useless_pages:
    for q in range(24):
        prdct=df_products[t][q]
        et_vrem_seg=prdct['ET_segments']
        for i in et_vrem_seg:
            if isinstance(i, np.int32):
                et_vrem_seg=np.delete(et_vrem_seg, np.where(et_vrem_seg==i)) #?#

In [ ]:
for t in useless_pages:
    for q in range(num_of_products):

        dfvrem_et=pd.DataFrame()
        prodct=df_products[t][q]
        product_info=prodct['ProductInfo']
        et_prodct=prodct['ET_segments']

        if len(et_prodct)==0:
            pass
        else:
            for i in range(len(et_chans)):
                location=et_chans[i]
                et_i=et_data[i]
                et_i_for_prodct=[]
                
                if isinstance(et_prodct[0], np.ndarray):
                    for j in et_prodct:
                        s=[]
                        for k in range(j[1]-j[0]+1):
                            s.append(et_i[j[0]+k-1])
                        et_i_for_prodct.append(s)
                else:
                    s=[]
                    for k in range(et_prodct[1]-et_prodct[0]+1):
                        s.append(et_i[et_prodct[0]+k-1])
                    et_i_for_prodct.append(s)
                dfvrem_et[f'{location}']=[et_i_for_prodct]
            dfvrem_et['info']=[product_info]
            dfvrem_et['product_page']=t
            dfvrem_et['product_number']=q
            df_et=pd.concat([df_et, dfvrem_et], ignore_index=True)
df_et.reset_index(inplace=True)

In [522]:
#df_et.to_csv(f'et_{file_name}.csv')